# Cloning the repo & Installing the pre-requisites

**Step-1: Clone the YOLOv5 repo from ultralytics**

In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16799, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 16799 (delta 185), reused 180 (delta 95), pack-reused 16459
Receiving objects: 100% (16799/16799), 15.58 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (11488/11488), done.


**Step-2: Installing the libraries**

In [1]:
!pip install -r /content/yolov5/requirements.txt

# Training YOLO on the coke-pepsi dataset

**Step-3: Upload the data downloaded from [roboflow](https://universe.roboflow.com/my-workspace-7m1hi/pepsi-and-cocacola-images/dataset/5) and unzip**

In [4]:
!unzip "/content/data" -d /content/data2/

Archive:  /content/data.zip
  inflating: /content/data2/README.dataset.txt  
  inflating: /content/data2/README.roboflow.txt  
  inflating: /content/data2/data.yaml  
   creating: /content/data2/train/
   creating: /content/data2/train/images/
 extracting: /content/data2/train/images/100_jpg.rf.69f36865a54932559f78b96fa351fdcf.jpg  
 extracting: /content/data2/train/images/100_jpg.rf.f0955cf89ca8d816a229a7a5cf8acdda.jpg  
 extracting: /content/data2/train/images/101_jpg.rf.517394298f973a068163a9f5bba64a63.jpg  
 extracting: /content/data2/train/images/101_jpg.rf.c7a8bac50cf3778f9eb70d60efbc8b3c.jpg  
 extracting: /content/data2/train/images/102_jpg.rf.1f28969358596efb68a637fa689091c4.jpg  
 extracting: /content/data2/train/images/102_jpg.rf.94a4942bd13848e621aef87012c3dd5b.jpg  
 extracting: /content/data2/train/images/103_jpg.rf.5af29da79dd77fe743a8afb1343cc376.jpg  
 extracting: /content/data2/train/images/103_jpg.rf.7438246e66ef2f02921173b28ef19dc1.jpg  
 extracting: /content/data2/

**Step-4: Training loop**

In [5]:
!python /content/yolov5/train.py --img 416 --batch 16 --epochs 10 --data /content/data2/data.yaml --weights yolov5s.pt

2024-07-17 11:45:07.770725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 11:45:07.770779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 11:45:07.772312: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/data2/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=F

**Step-5: Backup the weights**

In [14]:
!cp -r /content/yolov5 /content/drive/MyDrive/yolov5

# Perform the object detection on the video using the newly trained weights

**Step-6: Break video file into frames**

1. Accept the location of the video location

2. Extract the frames out of the video

3. Resize each frame to 416 x 416

4. Save each frame as [timestamp].jpeg in the provided output directory

In [15]:
import cv2
import os

def process_video(input_video_path, output_folder, width, height):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video {input_video_path}")
        return

    # Get the frames per second (fps) of the video
    fps = cap.get(cv2.CAP_PROP_FPS)

    frame_count = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Calculate the timestamp in seconds
        timestamp = frame_count / fps

        # Resize the frame
        resized_frame = cv2.resize(frame, (width, height))

        # Format the timestamp for the filename
        timestamp_str = f"{timestamp:.2f}".replace('.', '_')
        filename = f"{timestamp_str}.jpeg"
        output_path = os.path.join(output_folder, filename)

        # Save the frame as a JPEG file
        cv2.imwrite(output_path, resized_frame)

        frame_count += 1

    cap.release()
    print(f"Processing completed. Frames saved to {output_folder}")

# Example usage
input_video_path = '/content/drive/MyDrive/yolov5/data/video/videoplayback.mp4'
output_folder = '/content/drive/MyDrive/yolov5/data/frames'
width = 416
height = 416

process_video(input_video_path, output_folder, width, height)

Processing completed. Frames saved to /content/drive/MyDrive/yolov5/data/frames


**Step-7: Break directory containing ~21k frames into sub-directories containing 1k frames each**

In [33]:
import os
import shutil
import math

def split_directory(input_dir, output_dir, images_per_directory):
    # Create a list of all files in the input directory
    files = os.listdir(input_dir)

    # Calculate the number of directories needed
    num_directories = math.ceil(len(files) / images_per_directory)

    # Create output directories
    for i in range(num_directories):
        sub_dir_name = f'directory_{i+1}'
        output_sub_dir = os.path.join(output_dir, sub_dir_name)
        print('making:', output_sub_dir)
        os.makedirs(output_sub_dir, exist_ok=True)

    # Move files to respective directories
    count = 0
    current_dir_index = 0
    for file in files:
        if count == images_per_directory:
            current_dir_index += 1
            count = 0
        source_path = os.path.join(input_dir, file)
        dest_path = os.path.join(output_dir, f'directory_{current_dir_index + 1}', file)
        shutil.move(source_path, dest_path)
        count += 1

    print(f"Directory splitting completed into {num_directories} directories in {output_dir}.")

# Prompt user input for directories and number of images per directory
input_directory = "/content/drive/MyDrive/yolov5/data/frames"
output_directory = "/content/drive/MyDrive/yolov5/data/broken_frames"
images_per_directory = 1000

split_directory(input_directory, output_directory, images_per_directory)


making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_1
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_2
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_3
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_4
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_5
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_6
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_7
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_8
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_9
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_10
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_11
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_12
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_13
making: /content/drive/MyDrive/yolov5/data/broken_frames/directory_14
making: /content/drive/MyDriv

**Step-8: Generate predictions for ~1k at a time and write it to /content/drive/MyDrive/yolov5/results.txt (This is to not exceed the Colab compute limit)**

In [34]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_1" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_1, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_1/0_00.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_1/0_03.jpeg: 640x640 (no detections), 11.5ms
image 3/1000 /content/dr

In [35]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_2" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_2, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_2/33_37.jpeg: 640x640 1 pepsi, 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_2/33_40.jpeg: 640x640 2 pepsis, 11.6ms
image 3/1000 /content/drive/MyDrive/y

In [36]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_3" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_3, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_3/100_00.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_3/100_03.jpeg: 640x640 (no detections), 11.5ms
image 3/1000 /conten

In [38]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_4" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_4/100_10.jpeg: 640x640 (no detections), 11.6ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_4/100_13.jpeg: 640x640 (no detections), 11.6ms
image 3/1000 /conten

In [39]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_5" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_5, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_5/133_47.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_5/133_50.jpeg: 640x640 (no detections), 11.5ms
image 3/1000 /conten

In [40]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_6" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_6, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_6/166_83.jpeg: 640x640 (no detections), 11.5ms
WARNING ⚠️ NMS time limit 0.550s exceeded
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_6/166_87.jpeg: 640x640 1 

In [41]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_7" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_7, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_7/200_20.jpeg: 640x640 (no detections), 11.6ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_7/200_23.jpeg: 640x640 (no detections), 11.6ms
image 3/1000 /conten

In [42]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_8" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_8, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_8/233_57.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_8/233_60.jpeg: 640x640 (no detections), 11.5ms
image 3/1000 /conten

In [43]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_9" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_9, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_9/266_93.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_9/266_97.jpeg: 640x640 (no detections), 11.5ms
image 3/1000 /conten

In [44]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_10" --nosave --device 0

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_10/300_30.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_10/300_33.jpeg: 640x640 (no detections), 11.5ms
image 3/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_10/300_37.jpeg: 640x640 (no detections), 11.5ms
image 4/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_10/300_40.jpeg: 640x640 (no detections), 11.5ms
image 5/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_10/300_43.jpeg: 640x640 (no detections), 11.5ms
image 6/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_10/300_47.jpeg: 640x640 (no detections), 11.5ms
image 7/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_10/300_50.jpeg: 640x640 (no detections), 11.5ms
image 8/1000 /content/drive/MyDrive/yolov5/data/broken_frames/director

In [45]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_11" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_11, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_11/333_67.jpeg: 640x640 2 pepsis, 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_11/333_70.jpeg: 640x640 (no de

In [46]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_12" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_12, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_12/367_03.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_12/367_07.jpeg: 640x640 (no detections), 11.5ms
image 3/1000 /con

In [47]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_13" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_13, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_13/400_40.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_13/400_43.jpeg: 640x640 (no detections), 11.6ms
image 3/1000 /con

In [48]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_14" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_14, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_14/433_77.jpeg: 640x640 1 pepsi, 11.6ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_14/433_80.jpeg: 640x640 (no det

In [49]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_15" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_15, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_15/467_13.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_15/467_17.jpeg: 640x640 (no detections), 11.6ms
image 3/1000 /con

In [50]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_16" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_16, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_16/500_50.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_16/500_53.jpeg: 640x640 (no detections), 11.6ms
image 3/1000 /con

In [51]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_17" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_17, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_17/533_87.jpeg: 640x640 1 pepsi, 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_17/533_90.jpeg: 640x640 1 pepsi

In [52]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_18" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_18, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_18/567_23.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_18/567_27.jpeg: 640x640 (no detections), 11.5ms
image 3/1000 /con

In [53]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_19" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_19, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_19/600_60.jpeg: 640x640 1 pepsi, 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_19/600_63.jpeg: 640x640 1 pepsi

In [54]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_20" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_20, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_20/633_97.jpeg: 640x640 (no detections), 11.5ms
image 2/1000 /content/drive/MyDrive/yolov5/data/broken_frames/directory_20/634_00.jpeg: 640x640 (no detections), 11.5ms
image 3/1000 /con

In [55]:
!python /content/yolov5/detect.py --weight "/content/yolov5/runs/train/exp/weights/best.pt" --source "/content/drive/MyDrive/yolov5/data/broken_frames/directory_21" --nosave --device 0

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/yolov5/data/broken_frames/directory_21, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-342-g12be4996 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/279 /content/drive/MyDrive/yolov5/data/broken_frames/directory_21/667_33.jpeg: 640x640 (no detections), 11.5ms
image 2/279 /content/drive/MyDrive/yolov5/data/broken_frames/directory_21/667_37.jpeg: 640x640 (no detections), 11.5ms
image 3/279 /conten

**Step-9: Process results.txt and generate JSON file in the required format**

In [2]:
import json

def extract_jpeg_filename(input_string):
    # Find the start index of the .jpeg file path
    start_index = input_string.find('/content/drive/MyDrive/yolov5/data/broken_frames')

    if start_index == -1:
        return None  # Return None if the path is not found

    # Find the end index of the .jpeg file path
    end_index = input_string.find('.jpeg', start_index) + 5

    # Extract the file path
    file_path = input_string[start_index:end_index]

    # Extract the file name
    file_name = file_path.split('/')[-1]

    return file_name

pepsis = []
cokes = []
with open('/content/drive/MyDrive/yolov5/results.txt', 'r') as results:
  lines = results.readlines()
  for line in lines:
    if 'pepsi' in line:
      pepsis.append(extract_jpeg_filename(line).split('.')[0].replace('_', '.'))
    elif 'coke' in line:
      cokes.append(extract_jpeg_filename(line).split('.')[0].replace('_', '.'))

In [3]:
with open('/content/drive/MyDrive/yolov5/predictions.json', 'w') as obj:
  json.dump({'Pepsi_pts': pepsis,'CocaCols_pts': cokes}, obj)